# Import The Needed Packages

In [1]:
#Import Python Libraries
import numpy as np
import pandas as pd
import tweepy 
import csv
import re
import string
import nltk 
from nltk.tokenize import RegexpTokenizer # devide the words 

In [2]:
from credentials import *

# Connecting to Twitter API

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Reading The CSV File

In [4]:
#Read csv file      we have a new version dont use this file
df = pd.read_csv("/Users/saramuneef/Downloads/newExtractedData.csv")

In [36]:
#How much data do i have
len(df)

3680

In [37]:
#See part of the Data to inderstand the data
df.head()

,date,text,id,stance,location,clean text
0,3/7/2020,السلام عليكم هل اقدر ابيع جزء من اسهم اكتتاب ارامكو ؟ من تطبيق المباشر للافراد,1.236390e+18,Neutral,NaN,السلام عليكم هل اقدر ابيع جزء من اسهم اكتاب ارامكو من تطبيق المباشر لافراد
1,3/7/2020,#تاسي مساء الخير ... غداً تستأنف التداولات في السوق السعودي بعد فشل اتفاق اوبك بلس وهبوط اسعار النفط لمستويات متدنية جديدة * توقعاتكم للاسبوع المقبل ؟ كتوقع شخصي ١. كسر سعر اكتتاب ارامكو . ٢. كسر مستوى ٧٠٠٠ نقطة . ٣. هناك أسهم ستعاكس اتجاه السوق موفقين,1.236240e+18,Neutral,NaN,تاسي مساء الخير غداً تستانف التداولات في السوق السعودي بعد فشل اتفاق اوبك بلس وهبوط اسعار النفط لمستويات متدنيه جديده توقعاتكم لاسبوع المقبل كتوقع شخصي ١ كسر سعر اكتاب ارامكو ٢ كسر مستوي ٧٠ نقطه ٣ هناك اسهم ستعاكس اتجاه السوق موفقين
2,3/7/2020,والله يابو دانة لعنة انهيار السوق ٢٠٠٦ حتى اليوم ملاحقة سوقنا ، ومع اكتتاب ارامكو عدد كبير من الناس اكتتبو وصارو يتابعون السوق وأصبح في اهتمام في الفرص الي فيه وللأسف خابت توقعات الكثير منهم ، اعتقد نحتاج فتره زمنية اطول وتحسن الأوضاع الاقتصادية علشان نصل للمستويات الي ذكرها الاخ,1.236230e+18,Negative,Kingdom of Saudi Arabia,واله يابو دانه لعنه انهيار السوق ٢٠٦ حتي اليوم ملاحقه سوقنا ومع اكتاب ارامكو عد كبير من الناس اكتبو وصارو يتابعون السوق واصبح في اهتمام في الفرص الي فيه ولاسف خابت توقعات الكثير منهم اعتقد نحتاج فتره زمنيه اطول وتحسن الاوضاع الاقتصاديه علشان نصل لمستويات الي ذكرها الاخ
3,3/7/2020,استثمار مادي في العلاقه ! هذا اكتتاب ارامكو مو حب,1.236230e+18,Neutral,قضاعه,استثمار مادي في العلاقه هذا اكتاب ارامكو مو حب
4,3/7/2020,بالنسبة للميزانية السعوديه فقد خرجت من أزمة اسعار النفط على الأقل للخمس سنوات القادمه بسبب اكتتاب أرامكو و الخطه الاستثمارية الشجاعة التي اعتمدتها في السبع سنوات الاخيره وعقبال عندنا,1.236200e+18,Positive,NaN,بالنسبه لميزانيه السعوديه فقد خرجت من ازمه اسعار النفط علي الاقل لخمس سنوات القادمه بسب اكتاب ارامكو و الخطه الاستثماريه الشجاعه التي اعتمدتها في السبع سنوات الاخيره وعقبال عندنا


In [38]:
# Print a concise summary of a DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3680 entries, 0 to 3679
Data columns (total 6 columns):
date          3680 non-null object
text          3680 non-null object
id            3680 non-null float64
stance        3680 non-null object
location      2235 non-null object
clean text    3680 non-null object
dtypes: float64(1), object(5)
memory usage: 172.6+ KB


In [39]:
#Getting to know the Data type
df.dtypes

date          object 
text          object 
id            float64
stance        object 
location      object 
clean text    object 
dtype: object

In [40]:
# number of non-NA values
df.count()

date          3680
text          3680
id            3680
stance        3680
location      2235
clean text    3680
dtype: int64

In [41]:
# Retrieve list of columns
df.columns

Index(['date', 'text', 'id', 'stance ', 'location', 'clean text'], dtype='object')

# Data Cleaning

In [5]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [6]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [7]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [8]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [9]:
#start processing the tweet
def processPost(tweet):

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', str(tweet))
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',str(tweet))
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(str(tweet))
    
    # normalize the tweet
    tweet= normalize_arabic(str(tweet))
    
    # remove repeated letters
    tweet=remove_repeating_char(tweet)

    #Remove new lines with white space
    tweet = str(tweet).replace('\n',' ')
    tweet = str(tweet).replace('\t',' ')
    
    #remove emoji
    tweet = remove_emoji(tweet)
        
    return tweet

In [10]:
# remove Arabic stop words from the tweets
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
ar_stops = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saramuneef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# remove Arabic stop words from the tweets
for word in df['text']:
    if word in ar_stops:        
        words_filtered.remove(word)

In [12]:
df["clean text"] = df['text'].apply(lambda x: processPost(x))

In [14]:
dff =pd.read_csv("CLEANED_DATA1.csv")   

In [15]:
dff["clean text"] = df['text'].apply(lambda x: processPost(x))

In [13]:
pd.set_option('display.max_colwidth', -1)  
print(df.iloc[528])

date          18/01/2020 18:41                                                        
username      mmss60061                                                               
to            alsabban1982                                                            
replies       1                                                                       
retweets      0                                                                       
favorites     0                                                                       
text          اتوقع اننا في مرحلة الاندفاع .. خاصة بعد اكتتاب ارامكو ناس دخلت لأول مره
geo           NaN                                                                     
mentions      NaN                                                                     
hashtags      NaN                                                                     
id            1.2186e+18                                                              
permalink     https://twitter.com/mmss60061

# Drop Unwanted Colomns

In [14]:
# Drop The unwanted Colomns ()
temp = df.drop(['Unnamed: 0','username','to','replies','retweets','favorites','geo','mentions','hashtags','permalink'],axis=1)

NameError: name 'df' is not defined

# Save the Cleaned Data into a new CSV file

In [15]:
temp.to_csv("CLEANED DATA1.csv",index=False,encoding="utf-8-sig")

NameError: name 'temp' is not defined

In [53]:
df=pd.read_csv("newExtractedData2.csv")

FileNotFoundError: [Errno 2] File b'newExtractedData2.csv' does not exist: b'newExtractedData2.csv'

In [54]:
#The method that will get the location for the tweet it recevies tweet ID as parametr 
def setLocation(j):
    try:
        tweet=api.get_status(j)
        
        
    except:
        return None
    location = None
        

        # Try coordinates first, as it gives the most concrete data.
        
    if tweet.coordinates :
        location = (tweet.coordinates.coordinates).toString()
        
       # Next up, try tweet.place, which could also have a full street address, and will have at least the full_name and country, if not null 
    elif tweet.place :
        street = ""
        #if tweet.place.attributes.street_address:
         #   street += tweet.place.attributes.street_address + ", "
          #  if tweet.place.attributes['623:id']:
           #     street += tweet.place.attributes['623:id'] + ", "
             
        location = street + tweet.place.full_name + ", " + tweet.place.country
        # Lastly, if all else fails, grab the user location. This data is pretty poor, but is at least something. You may not want it, depending on your usage.
    elif tweet.user and tweet.user.location:
            location = tweet.user.location        
         
    return location

In [55]:
#Put the "permalink" colume in list
lin_data=df['permalink'].values.tolist()
newL=[]

KeyError: 'permalink'

In [56]:
#Cut the ID form the permalink and add it in a list
z=0
for l in lin_data:
    e=lin_data[z] +""
    s=e[-19:]
    index=s.find('/')
    if(index!= -1):
        s=s[index+1:]
    newL.append(s)
    z=z+1

NameError: name 'lin_data' is not defined

In [57]:
#Pass each ID to setLocation method and add the returned value in a list
newLocation=[]
for j in newL:
    newLocation.append(setLocation(j))
    


NameError: name 'newL' is not defined

In [58]:
#Add new colume(location)
df['location'] =newLocation

ValueError: Length of values does not match length of index

In [59]:
#Drop unwanted columes
temp = df.drop(['Unnamed: 0','username','to','replies','retweets','favorites','geo','mentions','hashtags','permalink'],axis=1)

KeyError: "['Unnamed: 0' 'username' 'to' 'replies' 'retweets' 'favorites' 'geo'\n 'mentions' 'hashtags' 'permalink'] not found in axis"

In [60]:
temp.to_csv("CLEANED DATA1.csv",index=False,encoding="utf-8-sig")

NameError: name 'temp' is not defined

In [16]:
#Add tokens
tokenizer = RegexpTokenizer(r'\w+')
dff["tokens"] = dff["clean text"].apply(tokenizer.tokenize)

In [17]:
dff.to_csv("final_cleaned.csv",index=False,encoding="utf-8-sig")

In [62]:
df.head()

,date,text,id,stance,location,clean text,tokens
0,3/7/2020,السلام عليكم هل اقدر ابيع جزء من اسهم اكتتاب ارامكو ؟ من تطبيق المباشر للافراد,1.236390e+18,Neutral,NaN,السلام عليكم هل اقدر ابيع جزء من اسهم اكتاب ارامكو من تطبيق المباشر لافراد,"[السلام, عليكم, هل, اقدر, ابيع, جزء, من, اسهم, اكتاب, ارامكو, من, تطبيق, المباشر, لافراد]"
1,3/7/2020,#تاسي مساء الخير ... غداً تستأنف التداولات في السوق السعودي بعد فشل اتفاق اوبك بلس وهبوط اسعار النفط لمستويات متدنية جديدة * توقعاتكم للاسبوع المقبل ؟ كتوقع شخصي ١. كسر سعر اكتتاب ارامكو . ٢. كسر مستوى ٧٠٠٠ نقطة . ٣. هناك أسهم ستعاكس اتجاه السوق موفقين,1.236240e+18,Neutral,NaN,تاسي مساء الخير غداً تستانف التداولات في السوق السعودي بعد فشل اتفاق اوبك بلس وهبوط اسعار النفط لمستويات متدنيه جديده توقعاتكم لاسبوع المقبل كتوقع شخصي ١ كسر سعر اكتاب ارامكو ٢ كسر مستوي ٧٠ نقطه ٣ هناك اسهم ستعاكس اتجاه السوق موفقين,"[تاسي, مساء, الخير, غدا, تستانف, التداولات, في, السوق, السعودي, بعد, فشل, اتفاق, اوبك, بلس, وهبوط, اسعار, النفط, لمستويات, متدنيه, جديده, توقعاتكم, لاسبوع, المقبل, كتوقع, شخصي, ١, كسر, سعر, اكتاب, ارامكو, ٢, كسر, مستوي, ٧٠, نقطه, ٣, هناك, اسهم, ستعاكس, اتجاه, السوق, موفقين]"
2,3/7/2020,والله يابو دانة لعنة انهيار السوق ٢٠٠٦ حتى اليوم ملاحقة سوقنا ، ومع اكتتاب ارامكو عدد كبير من الناس اكتتبو وصارو يتابعون السوق وأصبح في اهتمام في الفرص الي فيه وللأسف خابت توقعات الكثير منهم ، اعتقد نحتاج فتره زمنية اطول وتحسن الأوضاع الاقتصادية علشان نصل للمستويات الي ذكرها الاخ,1.236230e+18,Negative,Kingdom of Saudi Arabia,واله يابو دانه لعنه انهيار السوق ٢٠٦ حتي اليوم ملاحقه سوقنا ومع اكتاب ارامكو عد كبير من الناس اكتبو وصارو يتابعون السوق واصبح في اهتمام في الفرص الي فيه ولاسف خابت توقعات الكثير منهم اعتقد نحتاج فتره زمنيه اطول وتحسن الاوضاع الاقتصاديه علشان نصل لمستويات الي ذكرها الاخ,"[واله, يابو, دانه, لعنه, انهيار, السوق, ٢٠٦, حتي, اليوم, ملاحقه, سوقنا, ومع, اكتاب, ارامكو, عد, كبير, من, الناس, اكتبو, وصارو, يتابعون, السوق, واصبح, في, اهتمام, في, الفرص, الي, فيه, ولاسف, خابت, توقعات, الكثير, منهم, اعتقد, نحتاج, فتره, زمنيه, اطول, وتحسن, الاوضاع, الاقتصاديه, علشان, نصل, لمستويات, الي, ذكرها, الاخ]"
3,3/7/2020,استثمار مادي في العلاقه ! هذا اكتتاب ارامكو مو حب,1.236230e+18,Neutral,قضاعه,استثمار مادي في العلاقه هذا اكتاب ارامكو مو حب,"[استثمار, مادي, في, العلاقه, هذا, اكتاب, ارامكو, مو, حب]"
4,3/7/2020,بالنسبة للميزانية السعوديه فقد خرجت من أزمة اسعار النفط على الأقل للخمس سنوات القادمه بسبب اكتتاب أرامكو و الخطه الاستثمارية الشجاعة التي اعتمدتها في السبع سنوات الاخيره وعقبال عندنا,1.236200e+18,Positive,NaN,بالنسبه لميزانيه السعوديه فقد خرجت من ازمه اسعار النفط علي الاقل لخمس سنوات القادمه بسب اكتاب ارامكو و الخطه الاستثماريه الشجاعه التي اعتمدتها في السبع سنوات الاخيره وعقبال عندنا,"[بالنسبه, لميزانيه, السعوديه, فقد, خرجت, من, ازمه, اسعار, النفط, علي, الاقل, لخمس, سنوات, القادمه, بسب, اكتاب, ارامكو, و, الخطه, الاستثماريه, الشجاعه, التي, اعتمدتها, في, السبع, سنوات, الاخيره, وعقبال, عندنا]"


In [ ]:
temp.to_csv("CLEANED DATA1.csv",index=False,encoding="utf-8-sig")